In [2]:
import numpy as np
import pandas as pd

# df = pd.read_csv("FrankensteinFullForm.csv")
# df = df.as_matrix()
# print(df.shape)
# labels = pd.read_csv("LabelFrankensteinFullForm.csv", header = None)
# labels = labels.as_matrix()

<h1>Load Dataset</h1>

In [3]:
import pickle
from numpy import genfromtxt
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
import sklearn

TRAIN_RATIO = 0.8
AMOUNT_OF_CLASSES = 7


def loadDataset(path):
    inputdataset = (genfromtxt(path, delimiter=','))[1:,:]
    dataset = normalize(inputdataset)
    trainingAmount = int(TRAIN_RATIO * dataset.shape[0])
    training_dataset = dataset[0:trainingAmount,:]
    test_dataset = dataset[trainingAmount:,:]
    print("data: ", dataset.shape)
    return training_dataset.astype(float), test_dataset.astype(float)

def loadLabel(data_path, labels_path):
    dataset = (genfromtxt(data_path, delimiter=','))[1:,:]
#     dataset = normalize(dataset)
    trainingAmount = int(TRAIN_RATIO * dataset.shape[0])
    #     labels = pickle.load(INIT_PATH + path)
    with open(labels_path, 'rb') as handle:
        labels = pickle.load(handle)
    ###### TEMPORARY
    del labels[-1]
    ######
    all_labels = np.zeros((len(labels), AMOUNT_OF_CLASSES))
#     training_labels = np.zeros((trainingAmount, AMOUNT_OF_CLASSES))
#     test_labels = np.zeros((len(labels) - trainingAmount, AMOUNT_OF_CLASSES))
    label_counter = 0
    per_label = [0] * AMOUNT_OF_CLASSES
#     print(labels)
    for label in labels:
        tmp = np.zeros((1,AMOUNT_OF_CLASSES))
        if (label == "static"):
            tmp[0,0] = 1
            per_label[0] += 1
        elif (label == "move"):
            tmp[0,1] = 1
            per_label[1] += 1
        elif (label == "rot"):
            tmp[0,2] = 1
            per_label[2] += 1            
        elif (label == "frot"):
            tmp[0,3] = 1
            per_label[3] += 1
        elif (label == "light"):
            tmp[0,4] = 1
            per_label[4] += 1
        elif (label == "dynamiclight"):
            tmp[0,5] = 1
            per_label[5] += 1
        else:
            tmp[0,6] = 1
            per_label[6] += 1
        all_labels[label_counter,:] = tmp
#         if(label_counter < trainingAmount):
#             training_labels[label_counter,:] = tmp
#         else:
#             test_labels[label_counter - trainingAmount,:] = tmp
        label_counter += 1
    dataset, all_labels = shuffle(dataset, all_labels, random_state=0)
    return assignData(dataset, all_labels)
    
#     train_dataset = dataset[0:trainingAmount,:]
#     test_dataset = dataset[trainingAmount:,:]
#     print("data: ", dataset.shape)
    
#     train_labels = all_labels[:trainingAmount,:]
#     test_labels = all_labels[trainingAmount:,:]
    
#     print(", ".join([definition[i] + ": " + str(per_label[i]) for i in range (len(per_label))]))
#     return train_dataset.astype(float), test_dataset.astype(float),train_labels, test_labels

In [4]:
def assignData(data,labels):
    UNKNOWN_AMOUNT = 5
    train_amount = int(data.shape[0] * TRAIN_RATIO)
    AMOUNT_PER_CLASS = int((data.shape[0] - UNKNOWN_AMOUNT) * TRAIN_RATIO)
    class_count = [0] * train_amount
    train_data = []
    test_data = []
    train_labels = []
    test_labels = []

    for i in range (data.shape[0]):
        if(len(train_data) == train_amount):
            test_data.append(data[i])
            test_labels.append(labels[i])
        else:
            index = np.argmax(data[i])
            if(index == AMOUNT_OF_CLASSES - 1):
                test_data.append(data[i])
                test_labels.append(labels[i])
            if( class_count[index] < AMOUNT_PER_CLASS) :
                train_data.append(data[i])
                train_labels.append(labels[i])
                class_count[index] += 1
            else:
                test_data.append(data[i])
                test_labels.append(abels[i])
    return np.asarray(train_data), np.asarray(test_data), np.asarray(train_labels), np.asarray(test_labels)
                    

In [5]:
train_data, test_data, train_labels, test_labels = loadLabel("../unnormFrankensteinFullForm.csv","LabelFrankensteinFullForm.p")
print(train_labels.dtype)
print(train_data.shape)
print(test_data.shape)
print(train_labels.shape)
print(test_labels.shape)
# print(train_labels)
print(np.sum(train_labels,axis=0))

float64
(110, 10)
(28, 10)
(110, 7)
(28, 7)
[19. 16. 17. 17. 18. 18.  5.]


In [6]:
with open("../unnormFrankensteinFullForm.csv") as f:
    lines = (line for line in f if not line.startswith('#'))
    dataset = np.loadtxt(lines, delimiter=',', skiprows=1)
definition = ['static', 'move', 'rot', 'frot', 'light', 'dynamiclight','unknown']
scaler = StandardScaler(copy=True)
scaler.fit(dataset)

def normalize(dataset):
#     datasetTmp = np.copy(dataset)
#     maxx = np.max(dataset, axis=0)
#     minn = np.min(dataset, axis=0)
#     rangee = maxx - minn
#     datasetTmp -= minn
#     datasetTmp /= rangee
#     print("minn: ",minn)
#     print("rangee: ",rangee)
    datasetTmp = scaler.transform(dataset)
    return datasetTmp
dataset = normalize(dataset)

In [7]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
from keras import optimizers

/home/bon/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
model = Sequential()

model.add(Dense(7, activation='linear', input_dim=10))
model.add(Dense(128,activation='linear'))
model.add(Dense(256, activation='linear'))
model.add(Dense(128, activation='linear'))
model.add(Dense(64, activation='linear'))
model.add(Dense(7, activation='softmax'))

In [9]:
# opti = optimizers.SGD(lr=0.0002, decay=1e-6, momentum=0.9, nesterov=True)
opti =optimizers.Adam(beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='binary_crossentropy', optimizer=opti, metrics=['accuracy'])
# model.compile(loss='MSE', optimizer=opti, metrics=['accuracy'])

In [56]:
model.fit(normalize(train_data),train_labels,batch_size=1, epochs=100, verbose=1)

Epoch 1/100
110/110 [==============================] - 0s 1ms/step - loss: 0.2213 - acc: 0.9026
Epoch 2/100
110/110 [==============================] - 0s 1ms/step - loss: 0.2171 - acc: 0.9130
Epoch 3/100
110/110 [==============================] - 0s 2ms/step - loss: 0.2161 - acc: 0.9117
Epoch 4/100
110/110 [==============================] - 0s 2ms/step - loss: 0.2256 - acc: 0.9104
Epoch 5/100
110/110 [==============================] - 0s 2ms/step - loss: 0.2220 - acc: 0.9169
Epoch 6/100
110/110 [==============================] - 0s 1ms/step - loss: 0.2197 - acc: 0.9130
Epoch 7/100
110/110 [==============================] - 0s 1ms/step - loss: 0.2249 - acc: 0.9065
Epoch 8/100
110/110 [==============================] - 0s 1ms/step - loss: 0.2182 - acc: 0.9117
Epoch 9/100
110/110 [==============================] - 0s 1ms/step - loss: 0.2236 - acc: 0.9078
Epoch 10/100
110/110 [==============================] - 0s 1ms/step - loss: 0.2176 - acc: 0.9052
Epoch 11/100
110/110 [=================

In [10]:
score = model.evaluate(normalize(test_data), test_labels, verbose=0)
print(score)

[0.44818347692489624, 0.8520407676696777]


In [11]:
# serialize model to YAML
model_yaml = model.to_yaml()
with open("model10.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("model10.h5")
print("Saved model to disk")


Saved model to disk


In [15]:
from keras.models import model_from_yaml
# load YAML and create model
yaml_file = open('model20.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights("model20.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
# loaded_model.compile(loss='MSE', optimizer=opti, metrics=['accuracy'])
loaded_model.compile(loss='binary_crossentropy', optimizer=opti, metrics=['accuracy'])
# score = loaded_model.evaluate(test_data, test_labels, verbose=0)
score = loaded_model.evaluate(normalize(test_data), test_labels, verbose=0)

print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
acc: 90.31%


In [ ]:
definition = ['static', 'move', 'rot', 'frot', 'light', 'dynamiclight','unknown']

for test_num in range (10):
    lol = normalize(train_data[test_num].reshape(1,-1))
    lol_label = train_labels[test_num]
    # print(lol.shape)
    # print(lol, lol_label)
    prediction = loaded_model.predict(lol.reshape(1,10))
    print(definition[np.argmax(prediction[0])])
    print(definition[np.argmax(lol_label)])
    print("-----------------------------------------------")

In [25]:
pred = np.ndarray((train_data.shape[0]))
ground = np.ndarray((train_data.shape[0]))
for i in range (train_data.shape[0]):
    pred[i] = np.argmax(normalize(train_data[i].reshape(1,-1)))
    ground[i] = np.argmax(train_labels[i])

NameError: name 'createCSV' is not defined